In [1]:
from tqdm.notebook import tqdm
from datetime import datetime,timedelta
import calendar
from django.db.models import F,Q

In [120]:
road_companies = [x for x in Company.objects.filter(active=True).values_list('name',flat=True).order_by('name')]
road_companies

['Arteris Fernão Dias',
 'Arteris Fernão Dias - Conserva',
 'Arteris Fluminense',
 'Arteris Fluminense - Conserva',
 'Arteris Intervias',
 'Arteris Litoral Sul',
 'Arteris Litoral Sul - Conserva',
 'Arteris Litoral Sul - Faixa de Domínio',
 'Arteris Planalto Sul',
 'Arteris Planalto Sul - Conserva',
 'Arteris Planalto Sul - Faixa de Domínio',
 'Arteris Régis Bittencourt',
 'Arteris Régis Bittencourt - Conserva',
 'Arteris Régis Bittencourt - Faixa de Domínio',
 'Arteris Via Paulista',
 'Arteris Via Paulista - Faixa de Domínio',
 'Arteris Via Paulista - SISOBRAS/SISQUALI',
 'CAJ',
 'Caminhos da Serra Gaúcha',
 'CART',
 'Construtora Castilho',
 'Construtora Castilho (Ferrovias)',
 'Construtora Castilho (Grupo Pátria)',
 'Construtora Castilho (Prefeitura Curitiba)',
 'Dalba Foz do Iguaçu',
 'Eco Noroeste',
 'Eco Noroeste SISOAE',
 'Eixo SP',
 'Eixo SP - Fx. Dom. | Qualidade | Topografia',
 'Eixo SP - SISOAE',
 'ELO AutoBAn',
 'ELO EcoNoroeste',
 'ELO Ecopistas',
 'ELO Ecovias dos Imigrant

In [121]:
today = dt.date.today()
year = today.year
meses = [today.month - i for i in range(2)]
meses = [m if m > 0 else m + 12 for m in meses]  # Garante que os meses são válidos (1-12)
meses.sort()

months_number = meses
months_number

[2, 3]

In [122]:
last_days = [dt.date(year, a, calendar.monthrange(year, a)[1]).day for a in meses]
last_days

[29, 31]

# dias de uso

In [123]:
def dias_de_uso(months_name, company):
    qtd_dias = []
    dias = last_days
    
    for index, mes in enumerate(months_name):
        a = []
        for n in range(1, dias[index]):
            reporting_count = Reporting.objects.filter(
                company__name=company, 
                updated_at__year=year, 
                updated_at__month=mes,
                updated_at__day=n
            ).exclude(
                created_by__username='rlcs'
            ).exclude(
                created_by__username='neoviacs'
            ).exclude(
                created_by__username='arteriscs'
            ).only('uuid').count()
            
            a.append(reporting_count)
            
        qtd_dias.append(a)

    quantidade = []
    porcen_dias = []
    
    for qtd in qtd_dias:
        quantidade.append(len([x for x in qtd if x != 0]))
        
    for index, qtd in enumerate(quantidade):
        b = qtd / dias[index]
        porcen_dias.append(b)
        
    return porcen_dias


In [111]:
resultados_por_empresa = {}

for company in road_companies:
    resultados_por_empresa[company] = dias_de_uso(months_number, company)

# Imprimir os resultados por empresa
for company, resultados in resultados_por_empresa.items():
    print(f"Dias de uso para a empresa {company}: {resultados}")

Dias de uso para a empresa Arteris Fernão Dias: [0.8275862068965517, 0.0967741935483871]
Dias de uso para a empresa Arteris Fernão Dias - Conserva: [0.6896551724137931, 0.1935483870967742]
Dias de uso para a empresa Arteris Fluminense: [0.0, 0.0]
Dias de uso para a empresa Arteris Fluminense - Conserva: [0.20689655172413793, 0.03225806451612903]
Dias de uso para a empresa Arteris Intervias: [0.8620689655172413, 0.16129032258064516]
Dias de uso para a empresa Arteris Litoral Sul: [0.41379310344827586, 0.12903225806451613]
Dias de uso para a empresa Arteris Litoral Sul - Conserva: [0.7931034482758621, 0.1935483870967742]
Dias de uso para a empresa Arteris Litoral Sul - Faixa de Domínio: [0.034482758620689655, 0.0]
Dias de uso para a empresa Arteris Planalto Sul: [0.13793103448275862, 0.03225806451612903]
Dias de uso para a empresa Arteris Planalto Sul - Conserva: [0.6551724137931034, 0.12903225806451613]
Dias de uso para a empresa Arteris Planalto Sul - Faixa de Domínio: [0.0, 0.0]
Dias 

# w = variação número de apontamentos comparado ao mês anterior  

In [2]:
road_companies=['Eixo Sp - Pavimento']

In [11]:
today = datetime.now()-timedelta(days=20)
last_month = today - timedelta(days=30)
months = [last_month.month,today.month]
today,last_month,months

(datetime.datetime(2024, 2, 15, 17, 35, 10, 413530),
 datetime.datetime(2024, 1, 16, 17, 35, 10, 413530),
 [1, 2])

In [14]:
atual=Reporting.objects.filter(
                company__name = 'Eixo Sp - Pavimento', 
                updated_at__year=today.year, 
                updated_at__month=today.month
            ).exclude(
                created_by__username='rlcs'
            ).exclude(
                created_by__username='neoviacs'
            ).exclude(
                created_by__username='arteriscs'
            ).only('uuid').count()
anterior=Reporting.objects.filter(
                company__name = 'Eixo Sp - Pavimento', 
                updated_at__year=last_month.year, 
                updated_at__month=last_month.month
            ).exclude(
                created_by__username='rlcs'
            ).exclude(
                created_by__username='neoviacs'
            ).exclude(
                created_by__username='arteriscs'
            ).only('uuid').count()
atual/anterior-1,atual,anterior,anterior/atual-1

(0.6455696202531647, 650, 395, -0.39230769230769236)

# programações

In [ ]:
from django.db.models import Q
from datetime import datetime
 
mes_especifico = 2
ano_especifico = 2024
 
jobs_no_mes = Job.history.filter(
    Q(history_date__month=mes_especifico),
    Q(history_date__year=ano_especifico),
    company__name='Demo Concessionárias',
    history_type='+'
)
 
numero_de_jobs_no_mes = jobs_no_mes.count()
 
print(f'Número de jobs criados em fev de 2024: {numero_de_jobs_no_mes}')

# recursos

In [2]:
ProcedureResource.objects.filter(
    creation_date__month=2,
    creation_date__year=2024
).values(
    'firm__company__name',
    'creation_date__day'
).annotate(
qtd_dia=Count('uuid'))

<QuerySet [{'firm__company__name': 'Arteris Intervias', 'creation_date__day': 1, 'qtd_dia': 17}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 7, 'qtd_dia': 31}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 8, 'qtd_dia': 7}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 14, 'qtd_dia': 3}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 15, 'qtd_dia': 26}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 16, 'qtd_dia': 1}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 19, 'qtd_dia': 4}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 21, 'qtd_dia': 14}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 22, 'qtd_dia': 4}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 23, 'qtd_dia': 45}, {'firm__company__name': 'Arteris Intervias', 'creation_date__day': 26, 'qtd_dia': 3}, {'firm__company__name': 'Arteris Litoral 

# RDOs

In [3]:
from django.db.models import Q
from datetime import datetime
 
mes_especifico = 2  
ano_especifico = 2024
 
firms = Firm.objects.filter(company__name='Demo Concessionárias')
 
contador = 0
 
for firm in firms:
    if firm.firm_multiple_daily_report.filter(
        Q(date__month=mes_especifico),
        Q(date__year=ano_especifico)
    ).exists():
        contador += 1
 
porcentagem = (contador / firms.count()) * 100
 
print(f'Porcentagem de equipes com RDOs criados em fev de 2024: {porcentagem:.2f}%')

Porcentagem de equipes com RDOs criados em fev de 2024: 11.11%
